# Machine Learning 2 - Final Project

## Learning Team 8 - MSDS PT2025B

* Borromeo, Chloe
* Sison, Florence
* Young, Daniel O.
* Dorado, Joshua Renzo

In [39]:
# Import modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
plt.rcParams['figure.figsize'] = (8,6)

In [40]:
# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

## A. PREPROCESSING

In [41]:
# Read the Credit Card Balances data

credit_card = (
    pd.read_csv("./data/credit_card_balance.csv")
    # Keep only active contracts and with credit limit for upselling
    .query('NAME_CONTRACT_STATUS == "Active"')
    .sort_values(by=['SK_ID_CURR', 'MONTHS_BALANCE'], ascending=[False, False])
    .drop("NAME_CONTRACT_STATUS", axis=1)
    .set_index('SK_ID_CURR', drop=True)
)

credit_card["AMT_INST_MIN_REGULARITY"] = credit_card["AMT_INST_MIN_REGULARITY"].fillna(0)

display(credit_card.head())
credit_card.info(verbose=True, show_counts=True)

,SK_ID_PREV,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,...,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,SK_DPD,SK_DPD_DEF
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
456250,1794451,-1,153832.725,175500,0.0,0.0,0.0,0.0,8286.390,9675.0,...,151808.220,153832.725,153832.725,0.0,0,0.0,0.0,10.0,0,0
456250,1794451,-2,158266.935,175500,0.0,0.0,0.0,0.0,8477.730,9675.0,...,156053.160,158266.935,158266.935,0.0,0,0.0,0.0,9.0,0,0
456250,1794451,-3,162425.565,175500,0.0,0.0,0.0,0.0,8643.600,9000.0,...,160554.375,162425.565,162425.565,0.0,0,0.0,0.0,8.0,0,0
456250,1794451,-4,166188.150,180000,0.0,0.0,0.0,0.0,8804.565,9000.0,...,163871.775,166188.150,166188.150,0.0,0,0.0,0.0,7.0,0,0
456250,1794451,-5,169443.855,180000,0.0,0.0,0.0,0.0,8931.420,9000.0,...,167091.345,169443.855,169443.855,0.0,0,0.0,0.0,6.0,0,0


<class 'pandas.core.frame.DataFrame'>
Index: 3698436 entries, 456250 to 100006
Data columns (total 21 columns):
 #   Column                      Non-Null Count    Dtype  
---  ------                      --------------    -----  
 0   SK_ID_PREV                  3698436 non-null  int64  
 1   MONTHS_BALANCE              3698436 non-null  int64  
 2   AMT_BALANCE                 3698436 non-null  float64
 3   AMT_CREDIT_LIMIT_ACTUAL     3698436 non-null  int64  
 4   AMT_DRAWINGS_ATM_CURRENT    2976728 non-null  float64
 5   AMT_DRAWINGS_CURRENT        3698436 non-null  float64
 6   AMT_DRAWINGS_OTHER_CURRENT  2976728 non-null  float64
 7   AMT_DRAWINGS_POS_CURRENT    2976728 non-null  float64
 8   AMT_INST_MIN_REGULARITY     3698436 non-null  float64
 9   AMT_PAYMENT_CURRENT         2957056 non-null  float64
 10  AMT_PAYMENT_TOTAL_CURRENT   3698436 non-null  float64
 11  AMT_RECEIVABLE_PRINCIPAL    3698436 non-null  float64
 12  AMT_RECIVABLE               3698436 non-null  float64
 13

In [42]:
credit_card.index.nunique()

103468

In [43]:
print(credit_card.columns)

Index(['SK_ID_PREV', 'MONTHS_BALANCE', 'AMT_BALANCE',
       'AMT_CREDIT_LIMIT_ACTUAL', 'AMT_DRAWINGS_ATM_CURRENT',
       'AMT_DRAWINGS_CURRENT', 'AMT_DRAWINGS_OTHER_CURRENT',
       'AMT_DRAWINGS_POS_CURRENT', 'AMT_INST_MIN_REGULARITY',
       'AMT_PAYMENT_CURRENT', 'AMT_PAYMENT_TOTAL_CURRENT',
       'AMT_RECEIVABLE_PRINCIPAL', 'AMT_RECIVABLE', 'AMT_TOTAL_RECEIVABLE',
       'CNT_DRAWINGS_ATM_CURRENT', 'CNT_DRAWINGS_CURRENT',
       'CNT_DRAWINGS_OTHER_CURRENT', 'CNT_DRAWINGS_POS_CURRENT',
       'CNT_INSTALMENT_MATURE_CUM', 'SK_DPD', 'SK_DPD_DEF'],
      dtype='object')


#### A.1. Add new features to the data

With the help of some domain knowledge, we decided to engineer the following features here. 

In [44]:
# Monthly utilization
credit_card["Monthly_Utilization"] = credit_card["AMT_BALANCE"] / \
    credit_card["AMT_CREDIT_LIMIT_ACTUAL"]

# Tag if the customer paid in full
credit_card["Fully_Paid"] = (credit_card["AMT_BALANCE"] == 0)

# Ratio of the payment with the minimum payment
credit_card["Payment_MinPayment_Ratio"] = (
    credit_card['AMT_PAYMENT_TOTAL_CURRENT'] / credit_card['AMT_INST_MIN_REGULARITY']).fillna(0)

# Indicator if a client only paid the minimum amount
credit_card['Paid_Minimum_Only'] = (credit_card['AMT_INST_MIN_REGULARITY'] ==
                                    credit_card['AMT_PAYMENT_TOTAL_CURRENT']) & (credit_card['AMT_INST_MIN_REGULARITY'] > 0)

# Ratio of drawings from 'Other', 'ATM', 'POS' with Total Current Drawings in terms of transaction values
credit_card['AMT_Other_Ratio'] = (credit_card['AMT_DRAWINGS_OTHER_CURRENT'] /
                                  credit_card['AMT_DRAWINGS_CURRENT']).fillna(0)  # Drawings from Other Channels Ratio
credit_card['AMT_ATM_Ratio'] = (credit_card['AMT_DRAWINGS_ATM_CURRENT'] /
                                credit_card['AMT_DRAWINGS_CURRENT']).fillna(0)  # Drawings from ATM Ratio
credit_card['AMT_POS_Ratio'] = (credit_card['AMT_DRAWINGS_POS_CURRENT'] /
                                credit_card['AMT_DRAWINGS_CURRENT']).fillna(0)  # Drawings or usage using POS Terminals

# Ratio of drawings from 'Other', 'ATM', 'POS' with Total Current Drawings in terms of counts

# Count of drawings from Other Channels Ratio
credit_card['CNT_Other_Ratio'] = (
    credit_card['CNT_DRAWINGS_OTHER_CURRENT'] / credit_card['CNT_DRAWINGS_CURRENT']).fillna(0)
credit_card['CNT_ATM_Ratio'] = (credit_card['CNT_DRAWINGS_ATM_CURRENT'] /
                                credit_card['CNT_DRAWINGS_CURRENT']).fillna(0)  # Count of drawings from ATM Ratio

# Count of drawings or usage using POS Terminals
credit_card['CNT_POS_Ratio'] = (
    credit_card['CNT_DRAWINGS_POS_CURRENT'] / credit_card['CNT_DRAWINGS_CURRENT']).fillna(0)

#### A.2. Client-Level Aggregations

In [45]:
# Get the latest credit limit 
latest_credit_limits = (
    credit_card
    .sort_values(['MONTHS_BALANCE'])
    .groupby('SK_ID_CURR')
    .last()["AMT_CREDIT_LIMIT_ACTUAL"]
)

# Apply the aggregations
agg_functions = {
    'MONTHS_BALANCE': ['count'],
    'AMT_BALANCE': ['mean'],
    'Monthly_Utilization': ['mean', 'max'],
    'Fully_Paid': ['mean', 'sum'],
    'Paid_Minimum_Only': ['mean', 'sum'],
    'Payment_MinPayment_Ratio': ['mean'],
    'AMT_Other_Ratio': ['mean'],
    'AMT_ATM_Ratio': ['mean'],
    'AMT_POS_Ratio': ['mean'],
    'CNT_Other_Ratio': ['mean'],
    'CNT_ATM_Ratio': ['mean'],
    'CNT_POS_Ratio': ['mean']
}

credit_card_agg = (
    credit_card
    .groupby("SK_ID_CURR")
    .agg(agg_functions)
    .fillna(0)
    .replace([np.inf, -np.inf], 0)
)

# Rename columns
credit_card_agg.columns = [f'{col[0]}_{col[1]}' for col in credit_card_agg.columns]
credit_card_agg = credit_card_agg.rename(columns={
    'MONTHS_BALANCE_count': 'Tenure_Months', 
    'AMT_BALANCE_mean': 'Avg_Monthly_Balance',
    'Monthly_Utilization_mean': 'Avg_Monthly_Utilization',
    'Monthly_Utilization_max': 'Max_Monthly_Utilization',
    'Fully_Paid_mean': 'Pct_Txns_Fully_Paid',
    'Fully_Paid_sum': 'Months_Fully_Paid',
    'Paid_Minimum_Only_mean': 'Pct_Txns_Minimum_Paid',
    'Paid_Minimum_Only_sum': 'Months_Minimum_Paid',
    'AMT_Other_Ratio_mean': 'Avg_AMT_Other_Ratio',
    'AMT_ATM_Ratio_mean': 'Avg_AMT_ATM_Ratio',
    'AMT_POS_Ratio_mean': 'Avg_AMT_POS_Ratio',
    'CNT_Other_Ratio_mean': 'Avg_CNT_Other_Ratio',
    'CNT_ATM_Ratio_mean': 'Avg_CNT_ATM_Ratio',
    'CNT_POS_Ratio_mean': 'Avg_CNT_POS_Ratio'
})

# Incorporate the latest credit limit
credit_card_agg = pd.merge(credit_card_agg, latest_credit_limits,
                           how="inner", 
                           left_index=True, right_index=True)
credit_card_agg.head()


,Tenure_Months,Avg_Monthly_Balance,Avg_Monthly_Utilization,Max_Monthly_Utilization,Pct_Txns_Fully_Paid,Months_Fully_Paid,Pct_Txns_Minimum_Paid,Months_Minimum_Paid,Payment_MinPayment_Ratio_mean,Avg_AMT_Other_Ratio,Avg_AMT_ATM_Ratio,Avg_AMT_POS_Ratio,Avg_CNT_Other_Ratio,Avg_CNT_ATM_Ratio,Avg_CNT_POS_Ratio,AMT_CREDIT_LIMIT_ACTUAL
SK_ID_CURR,,,,,,,,,,,,,,,,
100006,6,0.000000,0.000000,0.00000,1.000000,6,0.000000,0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,270000
100011,74,54482.111149,0.302678,1.05000,0.554054,41,0.418919,31,0.502230,0.0,0.013514,0.0,0.0,0.013514,0.0,90000
100013,96,18159.919219,0.115301,1.02489,0.770833,74,0.000000,0,0.865673,0.0,0.052083,0.0,0.0,0.052083,0.0,45000
100021,7,0.000000,0.000000,0.00000,1.000000,7,0.000000,0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,675000
100023,8,0.000000,0.000000,0.00000,1.000000,8,0.000000,0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,225000


#### A.3. Merge with client-level information (e.g. Demographics and Properties Information)

In [46]:
# Read customer-related data

customer_1 = pd.read_csv("./data/application_train.csv")
customer_2 = pd.read_csv("./data/application_test.csv")

# Concatenate the DataFrames and include all specified columns
customer_info = (
    pd.concat([customer_1, customer_2], axis=0)
    # .set_index("SK_ID_CURR")
    .sort_index()
    [['SK_ID_CURR',  # Customer ID
      'TARGET',  # Tag with late payment
      'CODE_GENDER',  # Gender
      'FLAG_OWN_CAR',  # Owns a car
      'FLAG_OWN_REALTY',  # Owns real property
      'CNT_CHILDREN',  # Number of children
      'AMT_INCOME_TOTAL',  # Total income
      'NAME_INCOME_TYPE',  # Income type
      'NAME_EDUCATION_TYPE',  # Education type
      'NAME_FAMILY_STATUS',  # Family status
      'NAME_HOUSING_TYPE',  # Housing type
      'DAYS_BIRTH',  # Age
      'DAYS_EMPLOYED',  # Employment duration in days
      'OWN_CAR_AGE',  # Age of the car
      'OCCUPATION_TYPE',  # Occupation type
      'CNT_FAM_MEMBERS',  # Number of family members
      'EXT_SOURCE_1',  # External source 1
      'EXT_SOURCE_2',  # External source 2
      'EXT_SOURCE_3',  # External source 3

      'AMT_CREDIT',
      'AMT_ANNUITY',
      'AMT_GOODS_PRICE',

      # nums
      'FLAG_EMP_PHONE',
      'DAYS_REGISTRATION',
      'DAYS_ID_PUBLISH',
      'DEF_30_CNT_SOCIAL_CIRCLE',
      'DAYS_LAST_PHONE_CHANGE',
      # cats
      'REGION_RATING_CLIENT',
      'REG_CITY_NOT_LIVE_CITY',
      'REG_CITY_NOT_WORK_CITY',
      'NAME_CONTRACT_TYPE',
      ]]
)

# Rename column 'TARGET' with 'W_DEFAULT'
customer_info.rename(columns={'TARGET': 'W_DEFAULT'}, inplace=True)

customer_info.head()  


,SK_ID_CURR,W_DEFAULT,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,...,AMT_GOODS_PRICE,FLAG_EMP_PHONE,DAYS_REGISTRATION,DAYS_ID_PUBLISH,DEF_30_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,REGION_RATING_CLIENT,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,NAME_CONTRACT_TYPE
0,100002,1.0,M,N,Y,0,202500.0,Working,Secondary / secondary special,Single / not married,...,351000.0,1,-3648.0,-2120,2.0,-1134.0,2,0,0,Cash loans
0,100001,NaN,F,N,Y,0,135000.0,Working,Higher education,Married,...,450000.0,1,-5170.0,-812,0.0,-1740.0,2,0,0,Cash loans
1,100005,NaN,M,N,Y,0,99000.0,Working,Secondary / secondary special,Married,...,180000.0,1,-9118.0,-1623,0.0,0.0,2,0,0,Cash loans
1,100003,0.0,F,N,N,0,270000.0,State servant,Higher education,Married,...,1129500.0,1,-1186.0,-291,0.0,-828.0,1,0,0,Cash loans
2,100004,0.0,M,Y,Y,0,67500.0,Working,Secondary / secondary special,Single / not married,...,135000.0,1,-4260.0,-2531,0.0,-815.0,2,0,0,Revolving loans


In [47]:
# feature engineering
customer_info['CREDIT_TERM'] = customer_info.AMT_CREDIT/customer_info.AMT_ANNUITY
customer_info['CREDIT_INCOME_RATIO'] = customer_info.AMT_CREDIT/customer_info.AMT_INCOME_TOTAL
customer_info['ANNUITY_INCOME_RATIO'] = customer_info.AMT_ANNUITY/customer_info.AMT_INCOME_TOTAL
customer_info['CREDIT_GOODS_RATIO'] = customer_info.AMT_CREDIT/customer_info.AMT_GOODS_PRICE

In [48]:
# Join Credit Card data and Customer Information Dataframes

df_joined = pd.merge(customer_info, credit_card_agg, how="left", on='SK_ID_CURR')

# Express customer age in years
df_joined["Age_Years"] = abs(df_joined["DAYS_BIRTH"]) / 365

# Express customer age in years
df_joined["Employed_Months"] = abs(df_joined["DAYS_EMPLOYED"]) / 30

# Gender indicator
df_joined["CODE_GENDER_M"] = df_joined['CODE_GENDER'].map({'F': 0, 'M': 1})

# Indicator if client owns a car
df_joined["FLAG_OWN_CAR"] = df_joined['FLAG_OWN_CAR'].map({'Y': 1, 'N': 0})

# Indicator if client owns a real property
df_joined["FLAG_OWN_REALTY"] = df_joined['FLAG_OWN_REALTY'].map({'Y': 1, 'N': 0})

# Drop redundant columns
df_joined = df_joined.drop(['CODE_GENDER', 'DAYS_BIRTH'], axis=1)

# Drop rows with no value on any EXT_SOURCE_1, EXT_SOURCE_2, and EXT_SOURCE_3
external_sources = ['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']
# df_joined = df_joined[~df_joined[external_sources].isna().all(axis=1)]  # Only one row has none of all

# Create new columns to reflect best and worst scores from EXT_SOURCE_1, EXT_SOURCE_2, and EXT_SOURCE3
df_joined['EXT_SOURCE_best'] = df_joined[external_sources].apply(
    lambda row: row.max(skipna=True), axis=1)
df_joined['EXT_SOURCE_worst'] = df_joined[external_sources].apply(
    lambda row: row.min(skipna=True), axis=1)

# Isolate columns EXT_SOURCE_1, EXT_SOURCE_2, and EXT_SOURCE_3
df_joined = df_joined.drop(columns=external_sources)

In [49]:
# use mean to fill NA for own car age so that this data can be used without effect for the missing rows
df_joined['OWN_CAR_AGE'].fillna(df_joined.OWN_CAR_AGE.mean(), inplace=True)

# Create new feature credit limit to income ratio
df_joined['credit_limit_income_ratio'] = df_joined['AMT_CREDIT_LIMIT_ACTUAL'] / df_joined['AMT_INCOME_TOTAL']

In [50]:
# na_counts = df_joined['W_DEFAULT'].isna().sum()
# print(f"Number of NA values in W_DEFAULT column: {na_counts}")

df_joined

,SK_ID_CURR,W_DEFAULT,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,...,Avg_CNT_Other_Ratio,Avg_CNT_ATM_Ratio,Avg_CNT_POS_Ratio,AMT_CREDIT_LIMIT_ACTUAL,Age_Years,Employed_Months,CODE_GENDER_M,EXT_SOURCE_best,EXT_SOURCE_worst,credit_limit_income_ratio
0,100002,1.0,0,1,0,202500.0,Working,Secondary / secondary special,Single / not married,House / apartment,...,NaN,NaN,NaN,NaN,25.920548,21.233333,1.0,0.262949,0.083037,NaN
1,100001,NaN,0,1,0,135000.0,Working,Higher education,Married,House / apartment,...,NaN,NaN,NaN,NaN,52.715068,77.633333,0.0,0.789654,0.159520,NaN
2,100005,NaN,0,1,0,99000.0,Working,Secondary / secondary special,Married,House / apartment,...,NaN,NaN,NaN,NaN,49.490411,148.966667,1.0,0.564990,0.291656,NaN
3,100003,0.0,0,0,0,270000.0,State servant,Higher education,Married,House / apartment,...,NaN,NaN,NaN,NaN,45.931507,39.600000,0.0,0.622246,0.311267,NaN
4,100004,0.0,1,1,0,67500.0,Working,Secondary / secondary special,Single / not married,House / apartment,...,NaN,NaN,NaN,NaN,52.180822,7.500000,1.0,0.729567,0.555912,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356250,456251,0.0,0,0,0,157500.0,Working,Secondary / secondary special,Separated,With parents,...,NaN,NaN,NaN,NaN,25.553425,7.866667,1.0,0.681632,0.145570,NaN
356251,456252,0.0,0,1,0,72000.0,Pensioner,Secondary / secondary special,Widow,House / apartment,...,NaN,NaN,NaN,NaN,56.917808,12174.766667,0.0,0.115992,0.115992,NaN
356252,456253,0.0,0,1,0,153000.0,Working,Higher education,Separated,House / apartment,...,NaN,NaN,NaN,NaN,41.002740,264.033333,0.0,0.744026,0.218859,NaN
356253,456254,1.0,0,1,0,171000.0,Commercial associate,Secondary / secondary special,Married,House / apartment,...,NaN,NaN,NaN,NaN,32.769863,159.533333,0.0,0.661024,0.514163,NaN


In [51]:
df_joined.shape

(356255, 52)

#### A.4 Merge with installments data

In [52]:
installments_info = (
    pd.read_csv('./data/installments_payments.csv') 
    .sort_values(by=['SK_ID_CURR', 'NUM_INSTALMENT_NUMBER'], ascending=[False, False])
)

In [53]:
installments_info["LATE_PAYMENT"] = installments_info["DAYS_ENTRY_PAYMENT"] < installments_info["DAYS_INSTALMENT"]
installments_info["LATE_PAYMENT"] = installments_info["LATE_PAYMENT"].astype(int)

installments_info["DAYS_LATE"] = installments_info["DAYS_ENTRY_PAYMENT"] - installments_info["DAYS_INSTALMENT"]
installments_info["DAYS_LATE"] = np.abs(np.where(installments_info["DAYS_LATE"] < 0, installments_info["DAYS_LATE"], 0))

installments_info["AMT_LESS"] = np.abs(np.where(installments_info["AMT_PAYMENT"] < installments_info["AMT_INSTALMENT"], installments_info["AMT_PAYMENT"] - installments_info["AMT_INSTALMENT"], 0))

In [54]:
latest_installment = (
  installments_info
  .sort_values(['SK_ID_CURR', 'NUM_INSTALMENT_NUMBER'])
  .groupby('SK_ID_CURR')
  .last()['SK_ID_PREV']
)
latest_installment.head()
agg_functions = {
    'LATE_PAYMENT': ['sum'],
    'AMT_LESS': ['sum', 'max'],
    'DAYS_LATE': ['mean', 'max']
}

installments_agg = (
  installments_info
  .groupby("SK_ID_CURR")
  .agg(agg_functions)
  .fillna(0)
  .replace([np.inf, -np.inf], 0)
)
installments_agg.columns = [f'{col[0]}_{col[1]}' for col in installments_agg.columns]
installments_agg = installments_agg.rename(columns={
    'LATE_PAYMENT_sum': 'LATE_PAYMENT_DAYS', 
    'AMT_LESS_sum': 'AMT_LESS_TOTAL',
    'AMT_LESS_max': 'AMT_LESS_MAX',
    'DAYS_LATE_mean': 'DAYS_LATE_AVG',
    'DAYS_LATE_max': 'DAYS_LATE_MAX',
})
print(installments_agg.columns)

installments_agg = pd.merge(installments_agg, latest_installment,
                           how="inner", 
                           left_index=True, right_index=True)
installments_agg.head()

Index(['LATE_PAYMENT_DAYS', 'AMT_LESS_TOTAL', 'AMT_LESS_MAX', 'DAYS_LATE_AVG',
       'DAYS_LATE_MAX'],
      dtype='object')


,LATE_PAYMENT_DAYS,AMT_LESS_TOTAL,AMT_LESS_MAX,DAYS_LATE_AVG,DAYS_LATE_MAX,SK_ID_PREV
SK_ID_CURR,,,,,,
100001,4,0.0,0.0,8.857143,36.0,1851984
100002,19,0.0,0.0,20.421053,31.0,1038818
100003,25,0.0,0.0,7.160000,14.0,2396755
100004,3,0.0,0.0,7.666667,11.0,1564014
100005,8,0.0,0.0,23.666667,37.0,2495675


In [55]:
df_joined = pd.merge(df_joined, installments_agg, how='left', on='SK_ID_CURR')

df_joined.drop(columns=['SK_ID_PREV'], inplace=True, axis=1)

In [56]:
def fill_na():
  na_columns = df_joined.columns[df_joined.isna().any()].tolist()
  na_columns = [col for col in na_columns if col != 'OCCUPATION_TYPE']
  df_joined[na_columns] = df_joined[na_columns].fillna(0)

# fill_na()

#### A.5 Merge with POS_CASH data

In [57]:
pos_cash = (
    pd.read_csv('./data/POS_CASH_balance.csv') 
    .sort_values(by=['SK_ID_CURR'], ascending=[False])
) 

In [58]:
pos_cash_agg = pos_cash.groupby('SK_ID_CURR')['SK_DPD_DEF'].sum().reset_index()

df_joined = pd.merge(df_joined, pos_cash_agg, how='left', on='SK_ID_CURR')

In [59]:
df_joined['SK_DPD_DEF'].fillna(0, inplace=True)

#### A.6 Merge with previous application data

In [60]:
prev_app_info = (
  pd.read_csv('./data/previous_application.csv')
  .sort_values(by=['SK_ID_CURR'], ascending=[False])
)

In [61]:
prev_app_info['LESS_GRANTED'] = prev_app_info['AMT_APPLICATION'] > prev_app_info['AMT_CREDIT']
prev_app_info['REJECTED'] = prev_app_info['NAME_CONTRACT_STATUS'] == 'Refused'

In [62]:
prev_app_latest = (
  prev_app_info
  .sort_values(['SK_ID_CURR'],ascending=[False])
  .groupby('SK_ID_CURR')
  .last()['NAME_CONTRACT_TYPE']
)

In [63]:
prev_app_agg = (
  prev_app_info
  .groupby('SK_ID_CURR')
  .agg({
      'LESS_GRANTED': ['sum'],
      'REJECTED': ['sum'],
  })
  .fillna(0)
  .replace([np.inf, -np.inf], 0)
)

prev_app_agg.columns = [f'{col[0]}_{col[1]}' for col in prev_app_agg.columns]
prev_app_agg = prev_app_agg.rename(columns={
    'LESS_GRANTED_sum': 'LESS_GRANTED',
    'REJECTED_sum': 'REJECTED_APPLICATIONS'
})
prev_app_agg = pd.merge(prev_app_agg, prev_app_latest, how='inner', left_index=True, right_index=True)
prev_app_agg.drop(columns=['NAME_CONTRACT_TYPE'], inplace=True, axis=1)


In [64]:
df_joined = pd.merge(df_joined, prev_app_agg, how='left', on='SK_ID_CURR')

In [65]:
# fill_na()


In [66]:
df_joined.shape

(356255, 60)

#### A.7 Merge with bureau data

In [67]:
bureau_info = (
  pd.read_csv('./data/bureau.csv')
  .sort_values(by=['SK_ID_CURR'], ascending=[False])
)
bureau_info[['AMT_CREDIT_SUM', 'AMT_CREDIT_SUM_DEBT', 
       'AMT_CREDIT_SUM_LIMIT', 'AMT_CREDIT_SUM_OVERDUE']] = bureau_info[['AMT_CREDIT_SUM', 
       'AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM_LIMIT', 'AMT_CREDIT_SUM_OVERDUE']].fillna(0)

In [68]:
bureau_info['AMT_CREDIT_SUM_DEBT'] = bureau_info['AMT_CREDIT_SUM_DEBT'].fillna(0)
bureau_info['AMT_CREDIT_MAX_OVERDUE'] = bureau_info['AMT_CREDIT_MAX_OVERDUE'].fillna(0)


In [69]:
bureau_latest = (
  bureau_info
  .sort_values(['SK_ID_CURR'],ascending=[False])
  .groupby('SK_ID_CURR')
  .last()['CREDIT_ACTIVE']
)

bureau_agg = (
  bureau_info
  .groupby('SK_ID_CURR')
  .agg({
      'AMT_CREDIT_SUM': ['sum'],
      'AMT_CREDIT_SUM_DEBT': ['sum'],
      'AMT_CREDIT_SUM_LIMIT': ['sum'],
      'AMT_CREDIT_SUM_OVERDUE': ['sum'],
  })
  .fillna(0)
  .replace([np.inf, -np.inf], 0)
)

# Check for columns with missing values
bureau_agg.columns = [f'{col[0]}_{col[1]}' for col in bureau_agg.columns]
bureau_agg = bureau_agg.rename(columns={
    'AMT_CREDIT_SUM_sum': 'AMT_CREDIT_SUM_TOTAL',
    'AMT_CREDIT_SUM_DEBT_sum': 'AMT_CREDIT_SUM_DEBT_TOTAL',
    'AMT_CREDIT_SUM_LIMIT_sum': 'AMT_CREDIT_SUM_LIMIT_TOTAL',
    'AMT_CREDIT_SUM_OVERDUE_sum': 'AMT_CREDIT_SUM_OVERDUE_TOTAL',
})

# bureau_agg.drop(columns=['CREDIT_ACTIVE'], inplace=True, axis=1)
bureau_agg[['AMT_CREDIT_SUM_TOTAL', 'AMT_CREDIT_SUM_DEBT_TOTAL', 
            'AMT_CREDIT_SUM_LIMIT_TOTAL', 'AMT_CREDIT_SUM_OVERDUE_TOTAL']] = bureau_agg[['AMT_CREDIT_SUM_TOTAL', 
            'AMT_CREDIT_SUM_DEBT_TOTAL', 'AMT_CREDIT_SUM_LIMIT_TOTAL', 'AMT_CREDIT_SUM_OVERDUE_TOTAL']].fillna(0)

# Check for columns with missing values

In [70]:
df_joined = pd.merge(df_joined, bureau_agg, how='left', on='SK_ID_CURR')

In [71]:
minority_ratio = df_joined['W_DEFAULT'].value_counts()[1] / \
    df_joined['W_DEFAULT'].value_counts()[0]

print(f"Ratio of minority class to majority: {minority_ratio:.2f}")

Ratio of minority class to majority: 0.09


In [72]:
df_joined.to_csv('data/preprocessed.csv', index=False)
df_joined[df_joined.W_DEFAULT.notna()].to_csv('data/train_preprocessed.csv', index=False)
df_joined[df_joined.W_DEFAULT.isna()].to_csv('data/test_preprocessed.csv', index=False)


In [73]:
df_joined.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 356255 entries, 0 to 356254
Data columns (total 64 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   SK_ID_CURR                     356255 non-null  int64  
 1   W_DEFAULT                      307511 non-null  float64
 2   FLAG_OWN_CAR                   356255 non-null  int64  
 3   FLAG_OWN_REALTY                356255 non-null  int64  
 4   CNT_CHILDREN                   356255 non-null  int64  
 5   AMT_INCOME_TOTAL               356255 non-null  float64
 6   NAME_INCOME_TYPE               356255 non-null  object 
 7   NAME_EDUCATION_TYPE            356255 non-null  object 
 8   NAME_FAMILY_STATUS             356255 non-null  object 
 9   NAME_HOUSING_TYPE              356255 non-null  object 
 10  DAYS_EMPLOYED                  356255 non-null  int64  
 11  OWN_CAR_AGE                    356255 non-null  float64
 12  OCCUPATION_TYPE               